In [5]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [108 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [802 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [848 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [969 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,087 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,229 kB]
Hit:13 https://ppa.launchpad

In [6]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [7]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

In [8]:
homes_df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
homes_df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [9]:
# 2. Create a temporary view of the DataFrame.
homes_df.createOrReplaceTempView('home_sales')

In [10]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
a = """
SELECT
  YEAR(date) AS YEAR,
  ROUND(AVG(price), 2) AS AVERAGE_PRICE
FROM home_sales
WHERE bedrooms = 4
GROUP BY YEAR
ORDER BY YEAR DESC
"""

spark.sql(a).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [11]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
query1 = """
SELECT
date_built
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY date_built
ORDER BY date_built ASC
"""
spark.sql(query1).show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   292859.62|
|      2011|   291117.47|
|      2012|   293683.19|
|      2013|   295962.27|
|      2014|   290852.27|
|      2015|    288770.3|
|      2016|   290555.07|
|      2017|   292676.79|
+----------+------------+



In [12]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
query2 = """
SELECT
date_built
, round(avg(Price),2) as AveragePrice
from home_sales
WHERE bedrooms = 3
and bathrooms = 3
and floors = 2
and sqft_living >= 2000
GROUP BY date_built
ORDER BY date_built ASC
"""
spark.sql(query2).show()


+----------+------------+
|date_built|AveragePrice|
+----------+------------+
|      2010|   285010.22|
|      2011|   276553.81|
|      2012|   307539.97|
|      2013|   303676.79|
|      2014|   298264.72|
|      2015|   297609.97|
|      2016|    293965.1|
|      2017|   280317.58|
+----------+------------+



In [13]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 8.225440979003906e-05 seconds ---


In [14]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [15]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [16]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))


--- 7.224082946777344e-05 seconds ---


In [17]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy('date_built').mode('overwrite').parquet('parquet_home_sales')

NameError: ignored

In [18]:
# 11. Read the parquet formatted data.
pqdf = spark.read.parquet('parquet_home_sales')

AnalysisException: ignored

In [19]:
# 12. Create a temporary table for the parquet data.
pqdf.createOrReplaceTempView("p_home_sales")

NameError: ignored

In [20]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()



print("--- %s seconds ---" % (time.time() - start_time))

--- 6.818771362304688e-05 seconds ---


In [21]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [22]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
